In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from matplotlib import pyplot as plt
from typing import List
import numpy as np
import torch
import theseus as th
from nuplan_extent.planning.training.closed_loop.batched_nonlinear_smoother import BatchedNonLinearSmoother
from nuplan.planning.training.data_augmentation.data_augmentation_util import ConstrainedNonlinearSmoother
import gzip
import pickle
import time
import glob
import os
from pathlib import Path
import pandas

In [4]:
def get_random_perturb() -> List[float]:
    return [np.random.uniform(-1.5, 1.5), np.random.uniform(-1.5, 1.5), np.random.uniform(-np.pi/6, np.pi/6), np.random.uniform(0.0, 3.0)]

def get_th_input(x_curr, ref_traj: List[np.ndarray], device='cuda:0'):
    x_curr = torch.tensor(x_curr).to(torch.float).to(device)
    ref_traj = torch.stack([torch.from_numpy(i) for i in ref_traj], dim=0)
    bsize, _, dim = ref_traj.shape
    zeros = torch.zeros(bsize, 1, dim, dtype=ref_traj.dtype)
    ref_traj = torch.concat([zeros, ref_traj], dim=1).to(torch.float32).to(device)
    return x_curr, ref_traj

def get_cas_input(x_curr, ref_traj: List[np.ndarray]):
    ref_traj = [i.tolist() for i in ref_traj]
    ref_traj = [[[0., 0., 0.]] + i for i in ref_traj]
    return x_curr, ref_traj

def compute_ade(output_th, output_cas, gt):
    xy_th = output_th[:, :, :2].cpu().numpy()
    xy_cas = output_cas[:, :, :2]
    xy_gt = gt[:, :, :2]
    
    xy_diff_cas = xy_th - xy_cas
    xy_diff_gt = xy_th - xy_gt
    xy_diff_cas2gt = xy_cas - xy_gt
    
    result_cas = np.hypot(xy_diff_cas[..., 0], xy_diff_cas[..., 1]).mean() # reduced across batch and time dimension
    result_gt = np.hypot(xy_diff_gt[..., 0], xy_diff_gt[..., 1]).mean()
    result_cas2gt = np.hypot(xy_diff_cas2gt[..., 0], xy_diff_cas2gt[..., 1]).mean()
    return result_cas, result_gt, result_cas2gt

def compute_l1_fde(output_th, output_cas, gt):
    xy_th = output_th[:, -1, :2].cpu().numpy()
    xy_cas = output_cas[:, -1, :2] # [N, 2]
    xy_gt = gt[:, -1, :2]
    
    xy_diff_cas = xy_th - xy_cas
    xy_diff_gt = xy_th - xy_gt
    
    result_cas = np.hypot(xy_diff_cas[..., 0], xy_diff_cas[..., 1]).mean() # reduced across batch
    result_gt = np.hypot(xy_diff_gt[..., 0], xy_diff_gt[..., 1]).mean()
    
    return result_cas, result_gt
    
# For smoothness check, use square of jerk
# https://www.johndcook.com/blog/2018/03/30/curvature-and-automatic-differentiation/
def compute_smoothness(output_th, output_cas):
    def curvature(x, y):
        # Calculate the first and second derivatives
        dx = np.gradient(x)
        dy = np.gradient(y)
        d2x = np.gradient(dx)
        d2y = np.gradient(dy)

        # Calculate the curvature
        num = np.abs(d2x * dy - dx * d2y)
        den = (dx**2 + dy**2)**(3/2)
        k = num / den

        return k

    xy_th = output_th[:, :, :2].cpu().numpy()
    xy_cas = output_cas[:, :, :2]
    
    curv_th = np.zeros(xy_th.shape[0])
    for i in range(xy_th.shape[0]):
        c = curvature(xy_th[i, :, 0], xy_th[i, :, 1])
        curv_th[i] = np.mean(c)
    
    curv_cas = np.zeros(xy_cas.shape[0])
    for i in range(xy_cas.shape[0]):
        c = curvature(xy_cas[i, :, 0],xy_cas[i, :, 1])
        curv_cas[i] = np.mean(c)
        
    return curv_th, curv_cas

# Prepare data

In [5]:
data_root = Path("/mnt/nas20/nuplan_cached/trainval_closed_loop_raster6_no_overlap_1500_5Hz/")
meta_file = Path("/mnt/nas20/nuplan_cached/trainval_closed_loop_raster6_no_overlap_1500_5Hz_metadata/trainval_closed_loop_raster6_no_overlap_1500_5Hz_metadata_node_0.csv")

In [6]:
csv = pandas.read_csv(meta_file)

traj_files = [Path(i) for i in csv['file_name'].tolist() if 'trajectory' in i]

In [15]:
traj_med_v = [i.with_suffix('.gz') for i in traj_files if i.parents[2].stem == 'medium_magnitude_speed'][:80]
traj_high_v = [i.with_suffix('.gz') for i in traj_files if i.parents[2].stem == 'high_magnitude_speed'][:80]
traj_stationary = [i.with_suffix('.gz') for i in traj_files if i.parents[2].stem == 'stationary'][:80]
traj_chg_right = [i.with_suffix('.gz') for i in traj_files if i.parents[2].stem == 'changing_lane_to_right'][:80]
traj_right_turn = [i.with_suffix('.gz') for i in traj_files if i.parents[2].stem == 'starting_right_turn'][:80]

In [16]:
print(f"Length of traj_med_v: {len(traj_med_v)}")
print(f"Length of traj_high_v: {len(traj_high_v)}")
print(f"Length of traj_stationary: {len(traj_stationary)}")
print(f"Length of traj_chg_right: {len(traj_chg_right)}")
print(f"Length of traj_right_turn: {len(traj_right_turn)}")

Length of traj_med_v: 80
Length of traj_high_v: 80
Length of traj_stationary: 80
Length of traj_chg_right: 80
Length of traj_right_turn: 80


## Select the scenario type here.

In [18]:
def load_data(path):
    with gzip.open(path) as f:
        data = pickle.load(f)['data']
    return data

data = [load_data(i) for i in traj_right_turn]

In [19]:
perturb = [get_random_perturb() for _ in range(len(data))]
x_curr_th, ref_traj_th = get_th_input(perturb, data)
x_curr_cas, ref_traj_cas = get_cas_input(perturb, data)

batch_size = x_curr_th.shape[0]

print(f"x_curr_th shape {x_curr_th.shape}, ref_traj_th shape {ref_traj_th.shape}")
print(f"x_curr_cas shape {len(x_curr_cas)}")
print(f"batch_size {batch_size}")

x_curr_th shape torch.Size([80, 4]), ref_traj_th shape torch.Size([80, 17, 3])
x_curr_cas shape 80
batch_size 80


## Also prepare a version for drawing.

In [32]:
x_curr_draw = np.array(perturb)
ref_traj_draw = np.array(ref_traj_cas)

# Create smoothers

In [73]:
device = torch.device('cuda:0')

smoother = BatchedNonLinearSmoother(trajectory_len=16, dt=0.5, batch_size=80, device=device)

smoother2 = ConstrainedNonlinearSmoother(trajectory_len=16, dt=0.5)

## TH smoother experiment

In [74]:
output_th = smoother.solve(x_curr_th, ref_traj_th)

Nonlinear optimizer. Iteration: 0. Error: 1670.9130859375
Nonlinear optimizer. Iteration: 1. Error: 1356.08154296875
Nonlinear optimizer. Iteration: 2. Error: 1100.7777099609375
Nonlinear optimizer. Iteration: 3. Error: 893.7428588867188
Nonlinear optimizer. Iteration: 4. Error: 725.8604125976562
Nonlinear optimizer. Iteration: 5. Error: 589.7240600585938
Nonlinear optimizer. Iteration: 6. Error: 479.3312683105469
Nonlinear optimizer. Iteration: 7. Error: 389.8128356933594
Nonlinear optimizer. Iteration: 8. Error: 317.2215270996094
Nonlinear optimizer. Iteration: 9. Error: 258.35748291015625
Nonlinear optimizer. Iteration: 10. Error: 210.62442016601562
Nonlinear optimizer. Iteration: 11. Error: 171.91653442382812
Nonlinear optimizer. Iteration: 12. Error: 140.5272674560547
Nonlinear optimizer. Iteration: 13. Error: 115.07280731201172
Nonlinear optimizer. Iteration: 14. Error: 94.4308090209961
Nonlinear optimizer. Iteration: 15. Error: 77.69172668457031
Nonlinear optimizer. Iteration: 1

## CAS smoother experiment

In [35]:
output_cas = []
for x_curr, ref_traj in zip(x_curr_cas, ref_traj_cas):
    smoother2.set_reference_trajectory(x_curr, ref_traj)
    sol = smoother2.solve()
    output_temp = np.vstack(
        [
            sol.value(smoother2.position_x),
            sol.value(smoother2.position_y),
            sol.value(smoother2.yaw),
        ]
    ).T
    output_cas.append(output_temp)
output_cas = np.stack(output_cas, axis=0)

In [49]:
output_cas.shape

(80, 17, 3)

# Visualize

In [37]:
os.makedirs("pictures_right_turn")

In [75]:
for i in range(batch_size):
    plt.figure(figsize=(10,10))
    
    # GT
    plt.plot(ref_traj_draw[i, :, 0], ref_traj_draw[i, :, 1], 'o-', alpha=0.5)

    # theseus
    plt.plot(output_th[i, :, 0].cpu().numpy(), output_th[i, :, 1].cpu().numpy(), 'd-')
    # casadi
    plt.plot(output_cas[i, :, 0], output_cas[i, :, 1], 'x-')
    
    plt.axis('equal')
    plt.legend(['ref', 'theseus', 'casadi'])
    
    plt.plot(ref_traj_draw[i, 0, 0]+x_curr_draw[i, 0], ref_traj_draw[i, 0, 1]+x_curr_draw[i, 1], '1', color="cyan")
    plt.arrow(ref_traj_draw[i, 0, 0]+x_curr_draw[i, 0], ref_traj_draw[i, 0, 1]+x_curr_draw[i, 1], np.cos(x_curr_draw[i, 2]), np.sin(x_curr_draw[i, 2]))
    
    plt.savefig(f"pictures_right_turn/{i}.jpg")
    plt.close()

# Compute errors

## ADE

In [ ]:
# compute_ade(output_th, output_cas, ref_traj_draw)

## FDE

In [ ]:
# compute_l1_fde(output_th, output_cas, ref_traj_draw)

## Curvature

In [ ]:
# smoothness_th, smoothness_cas = compute_smoothness(output_th, output_cas)

# plt.plot(smoothness_th, '.-')
# plt.plot(smoothness_cas, '.-')
# plt.legend(['TH', 'CAS'])

# print(np.mean(smoothness_th))
# print(np.mean(smoothness_cas))

# Grid search of hard constraint weight

In [ ]:
# damping = [0.1, 0.5, 0.9]
# step_size = [0.1, 0.4, 0.5]
# boundaries = [10.0, 50.0, 100.0]

# from itertools import product

# min_ade = float('inf')
# min_set = None
# for d, ss, b in product(damping, step_size, boundaries):
#     smoother = BatchedNonLinearSmoother(trajectory_len=16, dt=0.5, batch_size=80, device=torch.device('cpu'), damping=d, step_size=ss, boundary_cost_weight=b)
    
#     output_th, _, _ = smoother.solve(x_curr_th.cpu(), ref_traj_th.cpu())
    
#     output_th = output_th[:, 8:, :]
    
#     ade = compute_ade(output_th, np.zeros_like(output_th), ref_traj_draw[:, 8:, :])
    
#     if ade[1] < min_ade:
#         min_ade = ade[1]
#         min_set = (d, ss, b)
    
#     print(f"damping: {d}, step: {ss}, boundary: {b}, ade: {ade}")